In [1]:
%pip freeze | Select-String "scikit-learn"

Note: you may need to restart the kernel to use updated packages.


'Select-String' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!python -V

Python 3.9.13


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']
def wrangle(year, month):
    url=f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet"
    df=pd.read_parquet(url)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    
    return df
    

    

In [6]:
year=2023
month=3
df=wrangle(year, month)
df.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration,ride_id
0,2,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.0,1.0,N,238,42,2,...,1.0,0.5,0.00,0.0,1.0,11.10,0.0,0.00,10.000000,2023/03_0
1,2,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.4,1.0,N,138,231,1,...,6.0,0.5,12.54,0.0,1.0,76.49,2.5,1.25,31.083333,2023/03_1
2,1,2023-03-01 00:15:04,2023-03-01 00:29:26,0.0,3.3,1.0,N,140,186,1,...,3.5,0.5,4.65,0.0,1.0,28.05,2.5,0.00,14.366667,2023/03_2


In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
predictions=y_pred

In [9]:
y_pred.std()

np.float64(6.247488852238703)

In [10]:
df_result=pd.DataFrame( {
    'ride_id': df['ride_id'],
    'prediction': predictions  
}
)
df_result.head()

,ride_id,prediction
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486


In [11]:
output_file = f'yellow_tripdata_{year}-{month:02d}_predictions.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',   
    compression=None,  
    index=False          
)

In [12]:
print(f"\nPredictions saved to {output_file}")


Predictions saved to yellow_tripdata_2023-03_predictions.parquet


In [13]:
import os 

In [14]:
file_size_bytes = os.path.getsize(output_file)
file_size_mb = file_size_bytes / (1024 * 1024) 

print(f"Size of the output file ({output_file}): {file_size_mb:.2f} MB")

Size of the output file (yellow_tripdata_2023-03_predictions.parquet): 65.46 MB
